In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds

In [10]:
dataset=tfds.load('judge-1377884607_tweet_product_company (2)')

ValueError: Parsing builder name string judge-1377884607_tweet_product_company (2).csv failed.
The builder name string must be of the following format:
  dataset_name[/config_name][:version][/kwargs]

  Where:

    * dataset_name and config_name are string following python variable naming.
    * version is of the form x.y.z where {x,y,z} can be any digit or *.
    * kwargs is a comma list separated of arguments and values to pass to
      builder.

  Examples:
    my_dataset
    my_dataset:1.2.*
    my_dataset/config1
    my_dataset/config1:1.*.*
    my_dataset/config1/arg1=val1,arg2=val2
    my_dataset/config1:1.2.3/right=True,foo=bar,rate=1.2


In [ ]:
train_dataset,test_datase=dataset['train'],dataset['test']

In [ ]:
batch size=32
train_dataset=train_dataset.shuffle(10000)
train_dataset=train_dataset.batch(batch_size)
test_dataset=test_dataset.batch(batch_size)


In [ ]:
encoder=tf.keras.layersTextVectorization()
encoder.adapt(train_dataset.map(lamda text,_:text))

In [ ]:
vocabulary=np.array(encoder.get_vocabulary())

In [ ]:
original_text=np.array(train_dataset.as_numpy_iterator())[0][0]
encoded_text=encoder(original_text)
decoded_text=''.join(vocabulary[encoded_text])

In [1]:
print('original text:',original_text)
print('encoded:',encoded_text)
print('decoded:',decoded_text)

In [ ]:
model=tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(len(encoder.get_vocabulary()),64,mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32))
    tf.keras.layers.Dense(64,activation='relu'),
    tf.keras.layers.Dense(1)
])

In [ ]:
model.summary()

In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logitis=True),
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

In [ ]:
history=model.fit(train_dataset,epochs=5,validatin_data=test_dataset)

In [ ]:
history_dict=history.history
acc=history_dict['accuracy']
val_acc=history_dict['val_accuracy']
print(acc)
print(val_acc)

In [ ]:
sample_review='The brand is good'
prediction=model.predict(np.array([sample_review]))
print(*prediction[0])

In [ ]:
if prediction[0]>0:
    print('positive')
else:
    print('negative')